# Section 10 : Generating E and B maps

While the data release includes I, Q and U maps we do not specifically release the E and B maps presented in DR4.  Here, we give an example as to how users can recover the E and B maps.

To return to the main notebook or to view a different section use the links in the table of contents.

## Table of contents

 --- 
> [Section 1](Section_1_Introduction.ipynb): Introduction to AdvACT Data Release 4

> [Section 2](Section_2_coadded_maps.ipynb): S08-18 Coadded maps from [Naess et al. (2020)](https://www.overleaf.com/5395816498bsknshgxbmyr) and an Introduction to Pixell

> [Section 3](Section_3_HEALPix_CAR_Conversions.ipynb):  Demonstration of methods to convert maps between HEALPix format and the CAR format.

> [Section 4](Section_4_visualize_objects.ipynb):  Guide to using Pixell to look at objects in the maps and stacking our maps on catalogue positions.

> [Section 5](Section_5_Lensing_maps.ipynb):  Introduction to the Lensing maps from [Darwish et al](https://arxiv.org/pdf/2004.01139.pdf)

> [Section 6](Section_6_delensing_param_shifts.ipynb):  Delensing the CMB powerspectrum and comparing the delensed and lensed parameters from [Han et al. 2020, in preparation]

> [Section 7](Section_7_Component_separated_maps.ipynb):  Guide to the use of the Component Separated maps from [Madhavacheril et al. (2020)](https://arxiv.org/abs/1911.05717)

> [Section 8](Section_8_individual_maps_and_harmonic_analysis.ipynb): Maps of Individual Patches and Harmonic Analysis using CAR maps [Aiola et al, in preparation]

> [Section 9](Section_9_Noise_sims.ipynb): Generating Noise Simulations 

> [Section 10](Section_10_E_B_maps.ipynb):  Look at how to generate E and B maps from I, Q, U maps. 

> [Section 11](Section_11_ACT_likelihood.ipynb): Using the ACT Likelihood code
 
---


## Initialize notebook

We begin by running notebook 2 which imports Pixell and other neccesary functions by running the introductory notebook and also imports the ACT coadded maps.


In [ ]:
%run Section_2_coadded_maps.ipynb

## Get E and B Maps
We start by reading in the source and source free maps needed for this process and then trimming them down to a smaller size.  From there we will use enmap's functions to convert the maps to spherical harmonics, and finally we
inverse Fourier transform the harmonic maps to get the E and B maps.

In [ ]:
# To begin with we need to add in the Q and U maps for the same coadded patch
# We then trim them, using submap, to the size of the small patch from earlier
box = np.array([[-28,40],[-12,-40]])*utils.degree
smap_I = smap_2.submap(box)

imap = enmap.read_map(path + "act_planck_s08_s18_cmb_f150_night_map_Q.fits")
smap_Q = imap.submap(box)

imap = enmap.read_map(path + "act_planck_s08_s18_cmb_f150_night_map_U.fits")
smap_U = imap.submap(box)

# Once we have them we will combine them into one object
smap_TQU = enmap.zeros((3,)+smap_I.shape, smap_I.wcs)
smap_TQU[0:3] = smap_I, smap_Q, smap_U

Since this process involves taking Fourier transformations of the maps we want to apply a mask that tapers the edges of the maps.  We can do so using the enmap.apod function which will force the maps to taper off to zero at the edges.

In [ ]:
# Define taper using enmap.apod and then plot the taper
apod_pix = 40 
taper = enmap.apod(smap_TQU*0+1,apod_pix) 
eshow(taper[0],  **{"downgrade": 2, "colorbar":True, "ticks": 2})

# Apply taper to the smap by multiplying the two maps together
smap_TQU *= taper

In [ ]:
# Here are the inital TQU maps with the tapers applied
eshow(smap_TQU, **{"downgrade": 2, "colorbar":True, "ticks": 1})
smap_TQU[1].wcs
smap_TQU.shape

The next step is to convert these to E and B and maps.  Enmap allows us to do so using the map2harm function.  We set the normalization to 'phys' for physical which normalizes the maps such that the binned square of the Fourier transformation can be directly compared to theory (apart from mask corrections).  

From there we combine all five maps (TQUEB) in a single map object them and plot them.

In [ ]:
# Apply Transformation to get T,E,B harmonic maps from the T,Q,U real-space maps
f_smap = enmap.map2harm(smap_TQU, normalize = "phys")

# Create a map object to hold all five maps
maps_TQUEB = enmap.zeros((5,)+smap_I.shape, smap_I.wcs)

# We now fill in the object.  The first three remain the same as the originals
# The second two are the EB maps which we get by inverse fourier transforming the 
# Harmonic maps in f_smap

maps_TQUEB[0:3] = smap_TQU
maps_TQUEB[3:]  = enmap.ifft(f_smap, normalize = "phys").real[1:]

In [ ]:
# Now we plot all five maps in order T, Q, U, E and the B

eshow(maps_TQUEB[0], **{"downgrade": 2, "ticks": 1, "color" : "planck", "range": 250})
eshow(maps_TQUEB[1:], **{"downgrade": 2, "ticks": 1, "color" : "gray", "range": 25})